In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/test_tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggi_fac")
secret_value_1 = user_secrets.get_secret("wandb")

%%capture is a magic command in Jupyter notebook that suppresses the output of the cell.

• %pip install -U transformers installs or upgrades the 'transformers' library, which is used for NLP tasks.

• %pip install -U datasets installs or upgrades the 'datasets' library, used for managing datasets.

• %pip install -U accelerate installs or upgrades the 'accelerate' library, used for distributed training.

• %pip install -U peft installs or upgrades the 'peft' library, which is a performance evaluation tool.

• %pip install -U trl installs or upgrades the 'trl' library, used for reinforcement learning tasks.

• %pip install -U bitsandbytes installs or upgrades the 'bitsandbytes' library, used for fast PyTorch operations.

• %pip install -U wandb installs or upgrades the 'wandb' library, used for experiment tracking.

In [3]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

The code begins by importing various modules from the transformers library.

• AutoModelForCausalLM is used for causal language modeling.

• AutoTokenizer is used to tokenize text data.

• BitsAndBytesConfig is a configuration class for the Bits and Bytes model.

• HfArgumentParser is a parser for command line arguments.

• TrainingArguments is used to define training parameters.

• pipeline is a high-level, easy to use, API for doing machine learning tasks. 

• logging is used for logging purposes.

• It then imports various modules from the peft library.

• LoraConfig is a configuration class for the Lora model.

• PeftModel is the main model class.

• prepare_model_for_kbit_training is a function to prepare the model for training.

• get_peft_model is a function to get the PEFT model.

• It imports the os module for interacting with the operating system.

• torch is imported for tensor computations and neural networks.

• wandb is imported for experiment tracking and visualization.

• load_dataset is imported from datasets to load datasets.

• SFTTrainer and setup_chat_format are imported from trl for training and setting up chat format.

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-07-24 01:28:10.233779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 01:28:10.233912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 01:28:10.356246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("huggi_fac")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on Medical Dataset', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ds4maths (ds4maths-indian-institute-of-technology-madras). Use `wandb login --relogin` to force relogin


In [6]:
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model = "llama-3-8b-chat-doctor"

The first line of code is defining a variable torch_dtype and assigning it the value torch.float16.

• torch.float16 is a data type in PyTorch, a machine learning library in Python.

• This data type is used for storing tensor elements in half precision floating point format.

• The second line of code is defining a variable attn_implementation and assigning it the string "eager".

• This could be a setting for how attention mechanisms are implemented in a neural network model.

In [7]:
torch_dtype = torch.float16
attn_implementation = "eager"

The code begins by setting up a configuration for QLoRA (Quantized Long Range Arena) using BitsAndBytesConfig.

• load_in_4bit=True indicates that the model weights will be loaded in 4-bit precision.

• bnb_4bit_quant_type="nf4" sets the type of 4-bit quantization to be used.

• bnb_4bit_compute_dtype=torch_dtype sets the data type for computations to be the same as torch_dtype.

• bnb_4bit_use_double_quant=True enables the use of double quantization.

• The AutoModelForCausalLM.from_pretrained method is then used to load a pre-trained model for causal language modeling.

• The base_model parameter specifies the model to be loaded.

• quantization_config=bnb_config applies the previously defined quantization configuration to the model.

• device_map="auto" allows the model to be automatically mapped to the available device (CPU or GPU).

• attn_implementation=attn_implementation sets the type of attention mechanism to be used in the model.

In [8]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

Adding the adapter to the layer

Fine-tuning the full model will take a lot of time, so to improve the training time, we’ll attach the adapter layer with a few parameters, making the entire process faster and more memory-efficient.

In [10]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

The dataset is imported using the load_dataset function, specifying the dataset name and split type.

• The dataset is shuffled with a seed of 65 and then the first 1000 samples are selected.

• A function format_chat_template is defined to format the data into a chat template.

• This function takes a row of data and creates a JSON object with the roles of "user" and "assistant".

• The content of the "user" and "assistant" roles are taken from the "Patient" and "Doctor" fields of the row.

• The chat template is then applied to the row using the apply_chat_template method of the tokenizer.

• The format_chat_template function is then applied to the entire dataset using the map function.

• The num_proc parameter is set to 4, which means that the mapping operation will be done in parallel using 4 processes.

• Finally, the fourth element of the 'text' column in the dataset is displayed.

In [11]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


'<|im_start|>user\nFell on sidewalk face first about 8 hrs ago. Swollen, cut lip bruised and cut knee, and hurt pride initially. Now have muscle and shoulder pain, stiff jaw(think this is from the really swollen lip),pain in wrist, and headache. I assume this is all normal but are there specific things I should look for or will I just be in pain for a while given the hard fall?<|im_end|>\n<|im_start|>assistant\nHello and welcome to HCM,The injuries caused on various body parts have to be managed.The cut and swollen lip has to be managed by sterile dressing.The body pains, pain on injured site and jaw pain should be managed by pain killer and muscle relaxant.I suggest you to consult your primary healthcare provider for clinical assessment.In case there is evidence of infection in any of the injured sites, a course of antibiotics may have to be started to control the infection.Thanks and take careDr Shailja P Wahal<|im_end|>\n'

In [12]:
dataset = dataset.train_test_split(test_size=0.1)

The code is creating an instance of the TrainingArguments class with specific parameters.

• output_dir=new_model sets the directory where the model and its metadata will be saved.

• per_device_train_batch_size=1 and per_device_eval_batch_size=1 set the batch size for training and evaluation.

• gradient_accumulation_steps=2 specifies the number of steps to accumulate gradients before updating parameters.

• optim="paged_adamw_32bit" sets the optimization algorithm to be used during training.

• num_train_epochs=1 sets the number of times the training loop will iterate over all training data.

• evaluation_strategy="steps" and eval_steps=0.2 define the evaluation to be done every 0.2 steps.

• logging_steps=1 and logging_strategy="steps" set the logging to be done every step.

• warmup_steps=10 sets the number of steps for the warm-up phase, where the learning rate is gradually increased.

• learning_rate=2e-4 sets the initial learning rate for the optimizer.

• fp16=False and bf16=False disable the use of 16-bit floating point precision during training.

• group_by_length=True enables the grouping of samples of similar length into the same batch.

• report_to="wandb" sets the reporting destination to Weights & Biases, a tool for tracking and visualizing machine learning experiments.

In [13]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


The code is initializing a trainer object from the SFTTrainer class.

• The model argument is the machine learning model that will be trained.

• train_dataset and eval_dataset are the training and testing datasets respectively.

• peft_config is the configuration for the Pretraining with Extracted Feature Tokens (PEFT).

• max_seq_length sets the maximum length of the sequences that the model will handle.

• dataset_text_field specifies the field in the dataset that contains the text to be processed.

• tokenizer is the tokenizer that will be used to process the text.

• args are the arguments for the training process.

• packing is a boolean that determines whether to pack multiple sequences together for efficiency.

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss,Validation Loss
90,1.921100,2.509598
180,2.394600,2.459979
270,2.346900,2.428414
360,2.455200,2.406353
450,2.489200,2.389822


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=450, training_loss=2.5913664221763613, metrics={'train_runtime': 1766.7287, 'train_samples_per_second': 0.509, 'train_steps_per_second': 0.255, 'total_flos': 9265767453646848.0, 'train_loss': 2.5913664221763613, 'epoch': 1.0})

In [16]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▅▃▂▁
eval/runtime,▂█▅▁▃
eval/samples_per_second,▁▁▁▁▁
eval/steps_per_second,▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▇▂▂▃▂▂▂▂▁▂▂▂▁▁▂▂▂▂▂▂▂▁▂▂▃▂▂▁▂▂▂▂▂▂▂▂▂▂▂
train/learning_rate,▄███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▃▃▂▄▃▃▂▃▂▂▃▃▂▂▂▃▃▃▂▃▃▂▃▂▁▃▂▂▂▃▂▂▂▂▂▂▂▂▂
eval/loss,2.38982
eval/runtime,75.6422


In [17]:
messages = [
    {
        "role": "user",
        "content": "Hello doctor, I have bad acne. How do I get rid of it?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Hello. I have gone through your query. I can understand your concern. Acne is a common problem. It can be treated with the help of medicines and lifestyle changes. I would suggest you to use a topical retinoid cream like Adapalene or Tretinoin. You can also use a topical antibiotic like Clindamycin or Erythromycin. You can also take oral antibiotics like Doxycycline or Minocycline. You can also take oral retinoids like Isotretinoin. You can also use a face wash containing salicylic acid. You can also


In [18]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Deepakiitm/llama-3-8b-chat-doctor/commit/0abe0a329d73de79273a9b8d51d058ad662bd6d8', commit_message='Upload model', commit_description='', oid='0abe0a329d73de79273a9b8d51d058ad662bd6d8', pr_url=None, pr_revision=None, pr_num=None)